## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-16-10-51-27 +0000,bbc,Federal court blocks Trump from sacking Lisa Cook,https://www.bbc.com/news/articles/c0ezqr7l774o...
1,2025-09-16-10-50-32 +0000,wapo,America and Britain were pillars of the world ...,https://www.washingtonpost.com/world/2025/09/1...
2,2025-09-16-10-49-14 +0000,bbc,Trump says he will sue New York Times for $15bn,https://www.bbc.com/news/articles/cevz419pk8vo...
3,2025-09-16-10-47-28 +0000,bbc,Kenya issues arrest warrant for British nation...,https://www.bbc.com/news/articles/cwywng4jp08o...
4,2025-09-16-10-44-26 +0000,bbc,Former Tory minister Maria Caulfield defects t...,https://www.bbc.com/news/articles/c1wg3zx4r0qo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2354/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,54
123,kirk,31
122,charlie,30
15,new,22
112,gaza,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
127,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...,121
83,2025-09-16-04-22-08 +0000,nypost,Old man who falsely claimed he shot Charlie Ki...,https://nypost.com/2025/09/16/us-news/old-man-...,115
116,2025-09-16-01-14-00 +0000,wsj,A federal appeals court on Monday night reject...,https://www.wsj.com/economy/central-banking/ap...,115
128,2025-09-16-00-39-31 +0000,nypost,Senate confirms Trump Fed pick Stephen Miran a...,https://nypost.com/2025/09/15/us-news/senate-c...,114
2,2025-09-16-10-49-14 +0000,bbc,Trump says he will sue New York Times for $15bn,https://www.bbc.com/news/articles/cevz419pk8vo...,113


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
127,121,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...
83,115,2025-09-16-04-22-08 +0000,nypost,Old man who falsely claimed he shot Charlie Ki...,https://nypost.com/2025/09/16/us-news/old-man-...
28,73,2025-09-16-09-45-00 +0000,wsj,Israel Launches New Ground Offensive Seeking t...,https://www.wsj.com/world/middle-east/israel-l...
116,61,2025-09-16-01-14-00 +0000,wsj,A federal appeals court on Monday night reject...,https://www.wsj.com/economy/central-banking/ap...
165,47,2025-09-15-22-18-14 +0000,nypost,"US, China near TikTok deal to finally resolve ...",https://nypost.com/2025/09/15/business/us-chin...
201,43,2025-09-15-20-15-10 +0000,nypost,Trump urges Republicans to back stopgap bill a...,https://nypost.com/2025/09/15/us-news/trump-ca...
286,42,2025-09-15-14-39-54 +0000,nypost,Tesla share surge as Elon Musk buys $1B in sto...,https://nypost.com/2025/09/15/business/tesla-s...
279,41,2025-09-15-15-15-00 +0000,wsj,Canadian factories notched the biggest rise in...,https://www.wsj.com/articles/canada-factory-an...
221,37,2025-09-15-18-54-19 +0000,latimes,Suspect sent text saying he would 'take out' C...,https://www.latimes.com/california/story/2025-...
114,35,2025-09-16-01-20-34 +0000,nypost,Evergreen High School student writes moving vi...,https://nypost.com/2025/09/15/us-news/evergree...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
